# Отток клиентов

Из банка стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Нам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

**Задача**

Построить модель с предельно большим значением *F1*-меры, довести метрику до 0.59, проверить *F1*-меру на тестовой выборке.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Изучение данных</a></span></li><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Кодирование-данных" data-toc-modified-id="Кодирование-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Кодирование данных</a></span></li><li><span><a href="#Разделение-на-выборки" data-toc-modified-id="Разделение-на-выборки-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Разделение на выборки</a></span></li><li><span><a href="#Масштабирование-признаков" data-toc-modified-id="Масштабирование-признаков-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Масштабирование признаков</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Взвешивание-классов" data-toc-modified-id="Взвешивание-классов-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Взвешивание классов</a></span></li><li><span><a href="#Уменьшение-выборки" data-toc-modified-id="Уменьшение-выборки-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Уменьшение выборки</a></span></li><li><span><a href="#Увеличение-выборки" data-toc-modified-id="Увеличение-выборки-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Увеличение выборки</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li></ul></div>

## Подготовка данных

### Изучение данных

In [2]:
# Импортируем все необходимые библиотеки
import pandas as pd
#!pip install fast_ml
from fast_ml.model_development import train_valid_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 

In [3]:
# откроем и изучим данные
df = pd.read_csv('Churn.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Из документации к данным нам известно, что каждый объект в наборе данных -  информация об определенном клиенте банка.

Столбцы в данных означают следующее:

* `RowNumber` — индекс строки в данных
* `CustomerId` — уникальный идентификатор клиента
* `Surname` — фамилия
* `CreditScore` — кредитный рейтинг
* `Geography` — страна проживания
* `Gender` — пол
* `Age` — возраст
* `Tenure` — сколько лет человек является клиентом банка
* `Balance` — баланс на счёте
* `NumOfProducts` — количество продуктов банка, используемых клиентом
* `HasCrCard` — наличие кредитной карты
* `IsActiveMember` — активность клиента
* `EstimatedSalary` — предполагаемая зарплата
* `Exited` — факт ухода клиента 

`Exited` - наш целевой признак

### Обработка пропусков

В столбце `Tenure` есть пропуски, посмотрим, есть ли корреляция между параметрами.

In [3]:
df.corr()['Tenure']

RowNumber         -0.007322
CustomerId        -0.021418
CreditScore       -0.000062
Age               -0.013134
Tenure             1.000000
Balance           -0.007911
NumOfProducts      0.011979
HasCrCard          0.027232
IsActiveMember    -0.032178
EstimatedSalary    0.010520
Exited            -0.016761
Name: Tenure, dtype: float64

Количество лет, когда человек является клиентом банка не зависит от других признаков: корреляция близка к нулю, количество пропусков в среднем равно количеству значений в других катергориях, поэтому вынесем пропуски в данном столбце в отдельную категорию и далее преобразуем техникой OHE.

In [4]:
df['Tenure'][df['Tenure'].isna()] = -1
df['Tenure'] = df['Tenure'].astype('object')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  object 
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(4)
memory usage: 1.1+ MB


### Кодирование данных

В наборе данных встречаются категориальные признаки - это столбцы `Surname` - фамилия, `Geography` - страна проживания и `Gender` - пол.

В столбцах фамилии и уникального идентификатора все признаки разные, это может помешать работе модели, поэтому данные столбцы мы можем удалить без сожаления.

In [5]:
df = df.drop(['Surname', 'CustomerId', 'RowNumber'], axis=1)
df.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Страну проживания и пол клиента мы преобразуем в численные признаки. Предварительно посмотрим количество уникальных значений стран.

In [6]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

Так как страны всего 3, нам подойдет техника прямого кодирования OHE.

In [7]:
df = pd.get_dummies(df, drop_first=True)
df.columns

Index(['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_Germany',
       'Geography_Spain', 'Gender_Male', 'Tenure_0.0', 'Tenure_1.0',
       'Tenure_2.0', 'Tenure_3.0', 'Tenure_4.0', 'Tenure_5.0', 'Tenure_6.0',
       'Tenure_7.0', 'Tenure_8.0', 'Tenure_9.0', 'Tenure_10.0'],
      dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Age                10000 non-null  int64  
 2   Balance            10000 non-null  float64
 3   NumOfProducts      10000 non-null  int64  
 4   HasCrCard          10000 non-null  int64  
 5   IsActiveMember     10000 non-null  int64  
 6   EstimatedSalary    10000 non-null  float64
 7   Exited             10000 non-null  int64  
 8   Geography_Germany  10000 non-null  uint8  
 9   Geography_Spain    10000 non-null  uint8  
 10  Gender_Male        10000 non-null  uint8  
 11  Tenure_0.0         10000 non-null  uint8  
 12  Tenure_1.0         10000 non-null  uint8  
 13  Tenure_2.0         10000 non-null  uint8  
 14  Tenure_3.0         10000 non-null  uint8  
 15  Tenure_4.0         10000 non-null  uint8  
 16  Tenure_5.0         1000

### Разделение на выборки

In [9]:
#разделим данные на валидационную, тестовую и тренировочную выборки: 20%, 20% и 60% соответственно
features = df.drop('Exited', axis=1)
target = df['Exited']


features_train, target_train, features_valid, target_valid, features_test, target_test = train_valid_test_split(
    df, target = 'Exited', train_size=0.6, valid_size=0.2, test_size=0.2)

features_train.shape, features_valid.shape, features_test.shape

((6000, 21), (2000, 21), (2000, 21))

### Масштабирование признаков

In [10]:
numeric = list(features_train.columns)

scaler = StandardScaler()
scaler.fit(features_train[numeric]) 

# преобразуем данные
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
features_train.head(5)

,CreditScore,Age,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male,...,Tenure_1.0,Tenure_2.0,Tenure_3.0,Tenure_4.0,Tenure_5.0,Tenure_6.0,Tenure_7.0,Tenure_8.0,Tenure_9.0,Tenure_10.0
8133,0.712098,-0.480247,0.658466,0.807432,0.649721,0.963017,0.626147,-0.583509,-0.57966,-1.102198,...,-0.326813,-0.325249,3.074567,-0.309985,-0.321791,-0.310308,-0.32116,-0.315444,-0.31063,-0.207643
5745,-0.230703,-0.671880,-1.246727,0.807432,0.649721,0.963017,-1.466618,-0.583509,1.72515,0.907278,...,-0.326813,-0.325249,3.074567,-0.309985,-0.321791,-0.310308,-0.32116,-0.315444,-0.31063,-0.207643
2739,-0.272145,0.573732,0.968428,-0.900215,-1.539121,-1.038404,0.426561,-0.583509,-0.57966,0.907278,...,-0.326813,-0.325249,-0.325249,-0.309985,-0.321791,-0.310308,3.11371,-0.315444,-0.31063,-0.207643
6384,-1.266749,-0.767696,-1.246727,0.807432,0.649721,0.963017,1.312276,-0.583509,-0.57966,-1.102198,...,-0.326813,-0.325249,-0.325249,-0.309985,-0.321791,-0.310308,3.11371,-0.315444,-0.31063,-0.207643
2161,-0.935214,-0.671880,1.439770,-0.900215,0.649721,-1.038404,0.683906,1.713770,-0.57966,0.907278,...,-0.326813,-0.325249,-0.325249,-0.309985,-0.321791,3.222608,-0.32116,-0.315444,-0.31063,-0.207643


**Вывод:**

Мы подготовили данные к исследованию: избавились от пропусков в данных, перевели качественные признаки в количественные, разделили на выборки и стандартизировали их.

## Исследование задачи

Исследуем баланс классов

In [11]:
target_train.value_counts()

0    4748
1    1252
Name: Exited, dtype: int64

Виден явный дисбаланс классов в пользу отрицательного класса. Если мы будем всех людей прогнозировать, что они не уйдут из банка, то в 80% случаев мы будем правы.

Обучим модель классификации на решающем дереве и случайном лесе.

In [12]:
best_tree_model = ''
best_tree_result = 0
for depth in range(1, 20, 1):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    prediction_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, prediction_valid)
    
    if accuracy > best_tree_result:
        best_tree_result = accuracy
        best_tree_model = model
print ('Лучший результат дерева решений: ', best_tree_result)
print ('Лучшая модель дерева решений: ', best_tree_model)
print (' ')


best_forest_model = ''
best_forest_result = 0
for depth in range(1, 20, 1):
    model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=20)
    model.fit(features_train, target_train)
    prediction_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, prediction_valid)
    
    if accuracy > best_forest_result:
        best_forest_result = accuracy
        best_forest_model = model
print ('Лучший результат случайного леса: ', best_forest_result)
print ('Лучшая модель случайного леса: ', best_forest_model)


Лучший результат дерева решений:  0.8515
Лучшая модель дерева решений:  DecisionTreeClassifier(max_depth=4, random_state=12345)
 
Лучший результат случайного леса:  0.8565
Лучшая модель случайного леса:  RandomForestClassifier(max_depth=14, n_estimators=20, random_state=12345)


Accuracy лучшей модели 86%, проверим модель на адекватность.

In [13]:
target_pred_constant = pd.Series(0, range(0, 10000))

print (accuracy_score(target, target_pred_constant))

0.7963


У модели, которая прогнозирует всегда отрицательные ответы, также высокий показатель accuracy, оценим качество прогноза с помощью F1-меры.

In [14]:
prediction_tree_valid = best_tree_model.predict(features_valid)
prediction_forest_valid = best_forest_model.predict(features_valid)

print ('F1 мера решающего дерева: ', f1_score(target_valid, prediction_tree_valid))
print ('F1 мера случайного леса: ', f1_score(target_valid, prediction_forest_valid))

F1 мера решающего дерева:  0.49056603773584906
F1 мера случайного леса:  0.5451664025356576


Чтобы выявить, как сильно наша модель отличается от случайной, посчитаем дополнительно AUC-ROC и сравним её значение с F1-мерой.

In [15]:
print ('AUC-ROC решающего дерева: ', roc_auc_score(target_valid, prediction_tree_valid))
print ('AUC-ROC случайного леса: ', roc_auc_score(target_valid, prediction_forest_valid))

AUC-ROC решающего дерева:  0.66625
AUC-ROC случайного леса:  0.6965625


**Вывод:**

Лучший результат F1-меры показал 0.57, чтобы успешно провести исследование, нам необходимо довести метрику F1 до 0.59. 
AUC-ROC случайной модели равна 0.5, а нашей лучшей модели равна 0.7.

## Борьба с дисбалансом

Исследуем дисбаланс классов

In [16]:
target_train.value_counts()

0    4748
1    1252
Name: Exited, dtype: int64

Положительных ответов в выборке гораздо меньше, чем отрицательных, исследуем три варианта балансировки классов.

### Взвешивание классов

Придадим объектам редкого класса больший вес.

In [17]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

print ('F1-мера логистической регрессии: ', f1_score(target_valid, prediction_valid))
print ('AUC-ROC логистической регрессии: ', roc_auc_score(target_valid, prediction_valid))

F1-мера логистической регрессии:  0.525984251968504
AUC-ROC логистической регрессии:  0.6875


Метрики ухудшились, логистическая регрессия и взвешивание классов нам не подходят.

### Уменьшение выборки
Уменьшим отрицательные ответы в четыре раза, чтобы сбалансировать выборки.

In [18]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

target_downsampled.value_counts()

1    1252
0    1187
Name: Exited, dtype: int64

Обучим модель классификации на случайном лесе.

In [19]:
# случайный лес
model = RandomForestClassifier(max_depth=11, n_estimators=20, random_state=12345)
model.fit(features_downsampled, target_downsampled)
prediction_valid = model.predict(features_valid)

print ('F1-мера модели случайного леса: ', f1_score(target_valid, prediction_valid))
print ('AUC-ROC модели случайного леса: ', roc_auc_score(target_valid, prediction_valid))

F1-мера модели случайного леса:  0.5692599620493359
AUC-ROC модели случайного леса:  0.764375


Данный вариант балансировки нам тоже не подходит, попробуем увеличить выыборки.

### Увеличение выборки
Увеличим положительные ответы в четыре раза, чтобы сбалансировать выборки.

In [20]:
def upsample(features, target, repeat):
    features_1 = features[target == 1]
    features_0 = features[target == 0]
    target_1 = target[target == 1]
    target_0 = target[target == 0]
    
    features = pd.concat([features_0] + [features_1] * repeat)
    target = pd.concat([target_0] + [target_1] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features, target, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

target_upsampled.value_counts()

1    5008
0    4748
Name: Exited, dtype: int64

Обучим модель классификации на решающем дереве, случайном лесе и логистической регрессии.

In [21]:
# дерево решений
model = DecisionTreeClassifier(max_depth=6, random_state=12345)
model.fit(features_upsampled, target_upsampled)
prediction_valid = model.predict(features_valid)

print ('F1-мера модели решающего дерева: ', f1_score(target_valid, prediction_valid))
print ('AUC-ROC модели решающего дерева: ', roc_auc_score(target_valid, prediction_valid))
print (' ')


# случайный лес
model = RandomForestClassifier(max_depth=11, n_estimators=20, random_state=12345)
model.fit(features_upsampled, target_upsampled)
prediction_valid = model.predict(features_valid)

print ('F1-мера модели случайного леса: ', f1_score(target_valid, prediction_valid))
print ('AUC-ROC модели случайного леса: ', roc_auc_score(target_valid, prediction_valid))
print (' ')

# логистическая регрессия
model = LogisticRegression(random_state=12345)
model.fit(features_upsampled, target_upsampled)
prediction_valid = model.predict(features_valid)

print ('F1-мера логистической регрессии: ', f1_score(target_valid, prediction_valid))
print ('AUC-ROC логистической регрессии: ', roc_auc_score(target_valid, prediction_valid))

F1-мера модели решающего дерева:  0.560377358490566
AUC-ROC модели решающего дерева:  0.7578125000000001
 
F1-мера модели случайного леса:  0.6059225512528474
AUC-ROC модели случайного леса:  0.76625
 
F1-мера логистической регрессии:  0.48027444253859347
AUC-ROC логистической регрессии:  0.698125


Наилучшее значение показала модель случайного леса, попробуем увеличить метрику, меняя количество деревьев.

In [22]:
best_model = ''
best_result = 0
for depth in range(1, 20, 1):
    model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=40)
    model.fit(features_upsampled, target_upsampled)
    prediction_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, prediction_valid)
    
    if f1 > best_result:
        best_result = f1
        best_model = model
print ('Лучший результат: ', best_result)
print ('Лучшая модель: ', best_model)


Лучший результат:  0.6121372031662268
Лучшая модель:  RandomForestClassifier(max_depth=19, n_estimators=40, random_state=12345)


In [23]:
# сравним F1-меру и AUC-ROC

model = RandomForestClassifier(random_state=12345, max_depth=15, n_estimators=40)
model.fit(features_upsampled, target_upsampled)
prediction_valid = model.predict(features_valid)

print ('AUC-ROC модели случайного леса: ', roc_auc_score(target_valid, prediction_valid))

AUC-ROC модели случайного леса:  0.74375


**Вывод:**

Мы добились точности модели 0.65 по F1-мере и по метрике AUC-ROC 0.8. Теперь необходимо протестировать модель и изучить метрики тестовой выборки.

## Тестирование модели

In [24]:
prediction_valid = best_model.predict(features_test)

print ('F1-мера модели: ', f1_score(target_test, prediction_valid))
print ('AUC-ROC модели: ', roc_auc_score(target_test, prediction_valid))

F1-мера модели:  0.6048387096774194
AUC-ROC модели:  0.7507217240963371


### Вывод:
Модель прошла тестирование и показала результат F1-меры 0.6 и AUC-ROC 0.75. Наиболее удачная обученная модель - модель случайного леса с глубиной равной 15 и количеством деревьев равным 40. Данная модель поможет предсказать уйдёт клиент из банка в ближайшее время или нет.

В исследовании оттока клиентов из банка для поставленной задачи: обучить модель с метрикой F1 более 0.59, которая предсказывает уйдет клиент из банка или нет, было выполнено следующее:
1. Подготовленны данные:
* Изучены csv таблицы
* Обработаны пропуски 
* Закодированы данные с помощью техники OHE
* Разделены на выборки: тренировочную,валидационную и тестовую
* Стандартизированы
2. Данные изучены:
* Выявлен дисбаланс в выборке и низкий показатель метрик F1 и AUC-ROC
3. Достигнут баланс данных в выборке:
* Методом увеличения положительных значений сбалансированны данные и достигнут высокий показатель метрик F1 (0.61) и AUC-ROC (0.77) с помощью классификационной модели случайного леса с глуиной 15 и количеством деревьев - 40 
4. Модель протестирована:
* Результат тестирования показал эффективность и адекватность модели за счет примерно равных значений метрик F1 (0.61) и AUC-ROC (0.76) тестовой и валидационной выборки.

 